In [78]:
#いろいろインポート
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

import datetime

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ

### dateのカラムから分布や時期の基礎集計と新規変数の検討

In [79]:
#モデルスコアデータの読み込み
place='asiya'
train_df = pd.read_csv('../csv/train_{}.csv'.format(place))
train_df=train_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
train_df['date']=pd.to_datetime(train_df['date'])#文字列なのでdateを日付型に変換
display(train_df)

,date,money,number_race,racer_1_ID,racer_1_age,racer_1_ave_st,racer_1_bo,racer_1_doub,racer_1_male,racer_1_mo,...,racer_5_rank,racer_6_ID,racer_6_age,racer_6_ave_st,racer_6_bo,racer_6_doub,racer_6_male,racer_6_mo,racer_6_rank,result_com
0,2012-04-03,1160.0,1,3959,35.0,0.14,3.0,0.541,1,43.0,...,2,4037,34.0,0.19,29.0,0.396,1,47.0,3,6
1,2012-04-03,2960.0,2,3251,46.0,0.18,10.0,0.450,1,36.0,...,3,4097,30.0,0.20,1.0,0.226,1,52.0,2,3
2,2012-04-03,21490.0,3,3662,43.0,0.16,4.0,0.480,1,68.0,...,4,3925,37.0,0.22,48.0,0.336,1,66.0,2,44
3,2012-04-03,5100.0,4,4266,27.0,0.17,21.0,0.506,1,46.0,...,2,4363,27.0,0.17,38.0,0.366,1,58.0,3,62
4,2012-04-03,1980.0,5,4112,32.0,0.21,40.0,0.439,1,39.0,...,2,4573,24.0,0.15,53.0,0.514,1,75.0,4,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19337,2020-03-27,54960.0,6,3503,51.0,0.19,26.0,0.156,1,44.0,...,2,3885,46.0,0.15,20.0,0.348,1,18.0,3,83
19338,2020-03-27,650.0,8,3873,44.0,0.17,41.0,0.426,1,61.0,...,4,3694,48.0,0.17,12.0,0.200,1,29.0,2,14
19339,2020-03-27,3930.0,9,3618,47.0,0.18,8.0,0.464,0,62.0,...,3,4252,37.0,0.17,18.0,0.279,1,23.0,2,10
19340,2020-03-27,1430.0,10,3710,46.0,0.17,45.0,0.247,1,52.0,...,2,4123,39.0,0.19,35.0,0.542,0,32.0,4,25


In [80]:
#新規変数の追加（月番号(month)、num_date,range_date）
#dateの型変換したものから月、年の情報の抽出
train_df['year']=train_df['date'].dt.year
train_df['month']=train_df['date'].dt.month
train_df['day']=train_df['date'].dt.day

#何日間の大会なのか(range_date)と大会(num_date)の何日目なのかの情報を追加

num_date=1
num_date_arr=[]
last_race_date=train_df['date'].values[0]#前レースの日付(処理開始時用にtarainのデータの一番初めのdateを仮に入力しておく)
for index,row in train_df.iterrows():
    today_date=row['date']
    if today_date==last_race_date:#同じ日のレースだったらおなじレース日を配列に追加、次の日の日付を出力（ほぼ無操作みたいなもん）
        next_date=row['date'] + datetime.timedelta(days=1)#次の日
        num_date_arr.append(num_date)
    else:#日にちが変わった時
        if today_date==next_date:#想定していた日付（次の日のレース）だったら,レース日を一日足して、そのレース日番号を加算
            num_date+=1
            num_date_arr.append(num_date)
            last_race_date=row['date']#前回レース日を上書き
            next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
        else:#想定していた日付でない(違う大会になった)場合はレース日をリセット
            num_date=1
            num_date_arr.append(num_date)
            last_race_date=row['date']#前回レース日を上書き
            next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
            
train_df['num_date']=num_date_arr
#train_df['range_date']=


In [81]:
range_races=0#大会中の取得できたレースの数
range_date=1#大会の開催日数
range_date_arr=[]
range_date_arr_2=[]#for文中で繰り返し上書きさせる用の配列
last_race_date=train_df['date'].values[0]#前レースの日付(処理開始時用にtrainのデータの一番初めのdateを仮に入力しておく)
for index,row in train_df.iterrows():
    today_date=row['date']
    if today_date==last_race_date:#同じ日のレースだったらおなじレース日を配列に追加、次の日の日付を出力（ほぼ無操作みたいなもん）
        range_races+=1
        next_date=row['date'] + datetime.timedelta(days=1)#次の日
        #num_date_arr.append(num_date)
    else:#日にちが変わった時
        if today_date==next_date:#想定していた日付（次の日のレース）だったら,レース日を一日足して終了
            range_date+=1
            range_races+=1
            last_race_date=row['date']#前回レース日を上書き
            next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日  
        else:#想定していた日付でない(違う大会になった)場合は現在のrange_dateをもとに前の大会のレースに大会開催日数を持たせる。

            range_date_arr_2=[range_date]*range_races
            for num in range_date_arr_2:
                range_date_arr.append(num)
            range_races=1#大会中の取得できたレースの数
            range_date=1#大会の開催日数
            last_race_date=row['date']#前回レース日を上書き
            next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
            
            
range_date_arr_2=[range_date]*range_races#最後の日は日付の変わり絵が発生しないので特別処理
for num in range_date_arr_2:
    range_date_arr.append(num)
train_df['range_date']=range_date_arr
#train_df['range_date']=


In [82]:
train_df.to_csv('../csv/train_{}_date.csv'.format(place),encoding='utf-8')

In [ ]:
#基礎集計(基本的な分布)

In [77]:
#train_df.to_csv('test.csv',encoding='utf-8')

In [17]:
next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]

0       2012-04-03
1       2012-04-03
2       2012-04-03
3       2012-04-03
4       2012-04-03
           ...    
19337   2020-03-27
19338   2020-03-27
19339   2020-03-27
19340   2020-03-27
19341   2020-03-27
Name: date, Length: 19342, dtype: datetime64[ns]

In [36]:
last_race_date=train_df['date'].values[0]
last_race_date

numpy.datetime64('2012-04-03T00:00:00.000000000')